In [2]:
import pandas as pd

In [29]:
headers = pd.read_csv('../Data/handles-data.csv')
tweets = pd.read_csv('../Data/tweets-data.csv')
union = pd.read_csv('../Data/union.csv', sep=";")
pd.options.display.float_format = '{:.2f}'.format

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Target

El target de nuestro proyecto va a ser el campo **nretweets** (número de Retweets).

He decidido usar este campo por su posible valor para conocer si el tuit va a tener la repercusión deseada o no.

He llegado a este conclusión después de ver cómo se comporta dependiendo de la hora y del día de la semana. Aunque no existen variaciones muy grandes, sí que he podido comprobar que ciertas horas y ciertos días hay más inteacción dentro de la red social.
Donde se puede ver que los usuarios dan más a me gusta y retuitean. Por lo que seguramente exista un patrón en los datos, dado un patrón de conducta humana. Por ejemplo, en las horas de "sueño" existe menos interacción.



# Limpieza de los Datos

## Eliminar No interesantes

In [4]:
print(tweets.columns.tolist())

['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']


Para el modelo, decido quedarme con las siguientes columnas:
- day
- hour
- followers

Considero que estos datos son los más interesantes, ya que para predecir podemos saber el día, la hora y los seguidores. Pero no otros datos como son el número de likes que tendrá, o si tendrá o no retweets.


In [30]:
datosInteresantes = union[['day','hour','followers']]
df_c = datosInteresantes.copy()
target = union[['nretweets']]
print(datosInteresantes.count())
print(target.count())

day          40867
hour         40867
followers    40867
dtype: int64
nretweets    40867
dtype: int64


### Nulos

In [31]:
datosInteresantes.isnull().sum()

day          0
hour         0
followers    0
dtype: int64

### Separamos datos categóricos

In [32]:
datosInteresantes.dtypes

day          int64
hour         int64
followers    int64
dtype: object

El campo retweet es categórico (Verdadero o Falso)

In [33]:
datosInteresantes['retweet']

KeyError: 'retweet'

In [16]:
from sklearn.preprocessing import OneHotEncoder

In [17]:
df_c = datosInteresantes.copy(deep = True)
enc_ohe = OneHotEncoder()
enc_ohe.fit(df_c[["retweet"]])

OneHotEncoder()

In [18]:
data_encoder = enc_ohe.transform(df_c[['retweet']]).toarray()
print(data_encoder)

[[0. 1.]
 [1. 0.]
 [0. 1.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


Lo unimos al dataset

In [19]:
df_encoder = pd.DataFrame(data_encoder)
df_encoder.head()
df_c = df_c.join(df_encoder,lsuffix='_left', rsuffix='_right')
df_c.drop(['retweet'], axis=1, inplace=True)


In [20]:
df_c

,day,hour,followers,0,1
0,4,0,20854298,0.00,1.00
1,2,22,108819032,1.00,0.00
2,2,18,108819032,0.00,1.00
3,2,18,108819032,1.00,0.00
4,2,7,108819032,1.00,0.00
...,...,...,...,...,...
40862,2,13,2003463,1.00,0.00
40863,2,13,2003463,1.00,0.00
40864,2,13,2003463,1.00,0.00
40865,2,13,2003463,1.00,0.00


## Separar datos Entrenamiento

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df_c, target, test_size=0.3)
print(X_train.shape)
print(X_test.shape)


(28606, 5)
(12261, 5)


## Nulos

In [23]:
X_train.isnull().sum()

day          0
hour         0
followers    0
0            0
1            0
dtype: int64

# Entrenar Modelo

In [24]:

import numpy as np
from sklearn.linear_model import LinearRegression

In [25]:
np.random.seed(5)


In [26]:
model=LinearRegression()


In [19]:
# X_train = np.asarray(X_train).astype(np.int64)

In [27]:
model.fit(X_train, y_train) 

/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [28]:

predictions = model.predict(X_test)
X_tested = X_test.copy()
print(X_tested.shape)
print(predictions.shape)
X_tested['predictions'] = predictions
X_tested['Target_test'] = y_test

# X_tested['difference'] = (y_test - X_tested['predictions'])
X_tested
# X_test['predictions'] = predictions
# X_test['defference'] = (y_test) - (X_test.predictions)

# día,hora,nreplies,nlikes,0_retweet,1_retweet
# print(model.predict([[6,3,0,0,1,0]]))

(12261, 5)
(12261, 1)


/home/julianmelero/twitter-ai/twai/lib/python3.8/site-packages/sklearn/utils/validation.py:1688: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


,day,hour,followers,0,1,predictions,Target_test
34507,3,4,16749405,1.00,0.00,401.76,170
34657,1,23,16749405,1.00,0.00,361.47,51
33113,4,5,3802924,1.00,0.00,139.36,3
24010,3,0,20147821,1.00,0.00,480.20,1942
31032,3,13,2114991,1.00,0.00,92.26,2
...,...,...,...,...,...,...,...
9713,5,16,6343145,1.00,0.00,137.28,7
16323,4,17,7212256,1.00,0.00,162.94,0
16140,6,11,3932614,1.00,0.00,96.05,10
7912,2,3,5072599,1.00,0.00,195.54,1


In [70]:
print(model.predict([[7,15,1,0]]))
X_tested.describe()

[[190.43616088]]


,day,hour,0,1,predictions,Target_test
count,12261.00,12261.00,12261.00,12261.00,12261.00,12261.00
mean,3.83,13.06,0.85,0.15,300.54,257.90
std,1.90,7.35,0.36,0.36,124.55,1545.44
min,1.00,0.00,0.00,0.00,157.80,0.00
25%,2.00,6.00,1.00,0.00,227.53,4.00
50%,4.00,15.00,1.00,0.00,260.90,16.00
75%,5.00,19.00,1.00,0.00,302.81,76.00
max,7.00,23.00,1.00,1.00,679.17,77827.00
